<a href="https://colab.research.google.com/github/Hallyeo/Data-Analysis-with-Open-Source/blob/main/%EC%A4%91%EA%B0%84%EA%B3%BC%EC%A0%9C%EB%AC%BC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import requests
import json
import time

# 1. 설정 변수 정의
# ----------------------------------------------------------------------
YOUR_API_KEY = "776164685877777738384575645171"
BASE_URL = "http://openapi.seoul.go.kr:8088/"
SERVICE_NAME = "energyUseDataSummaryInfo"
DATA_TYPE = "json"
START_INDEX = 1
END_INDEX = 100

# 요청할 데이터의 범위
START_YEAR = 2015
END_YEAR = 2024
# ----------------------------------------------------------------------

# 2. 데이터 수집 루프 (2015년 1월 ~ 2024년 12월 전체)
success_count = 0
fail_count = 0

for year in range(START_YEAR, END_YEAR + 1):
    for month in range(1, 13):

        # API 호출 URL 구성
        url = (f"{BASE_URL}/{YOUR_API_KEY}/{DATA_TYPE}/{SERVICE_NAME}/"
               f"{START_INDEX}/{END_INDEX}/")

        # 요청 매개변수 (YEAR, MON 인자 사용)
        params = {
            'YEAR': str(year),
            'MON': f"{month:02d}", # 두 자리 숫자로 포맷 (01~12)
        }

        print(f"\n📡 요청 시도: {year}년 {month:02d}월")

        try:
            # API 호출
            response = requests.get(url, params=params)

            # 3. HTTP 상태 코드 확인 (200이면 성공)
            if response.status_code == 200:
                data = response.json()

                # 서울시 API의 일반적인 성공 응답 확인
                if SERVICE_NAME in data:
                    rows = data[SERVICE_NAME].get('row', [])
                    print(f"✅ **성공.** HTTP 200, JSON 파싱 완료. 수집 데이터 {len(rows)}건 확인.")
                    success_count += 1
                elif 'RESULT' in data and data['RESULT'].get('CODE') in ['INFO-000', 'INFO-300']:
                    # INFO-000: 정상 처리되었으나 결과 데이터가 없음 (데이터는 없지만 호출은 성공)
                    print(f"ℹ️ **결과 없음/오류 코드 반환.** 응답 코드: {data['RESULT'].get('CODE')}")
                    success_count += 1 # 호출 자체는 성공으로 간주
                else:
                    print(f"❌ **파싱 실패/알 수 없는 응답.** HTTP 200이지만 예상치 못한 응답 구조.")
                    fail_count += 1

            else:
                print(f"❌ **API 호출 실패: HTTP 상태 코드 {response.status_code}**")
                fail_count += 1

        except requests.exceptions.RequestException as e:
            print(f"🚨 **네트워크 오류:** {e}")
            fail_count += 1
        except json.JSONDecodeError:
             print("❌ **JSON 파싱 오류.** 응답이 JSON 형식이 아님. 인증키나 URL을 확인하세요.")
             fail_count += 1
        except Exception as e:
            print(f"⚠️ **예상치 못한 오류:** {e}")
            fail_count += 1

        # API 부하 방지를 위한 짧은 대기 시간
        time.sleep(0.1)

# 4. 최종 결과 요약
print("\n========================================================")
print(f"🎉 **데이터 수집 상태 요약 (2015.01 ~ 2024.12)**")
print(f"✅ 성공적인 요청: {success_count}회")
print(f"❌ 실패한 요청: {fail_count}회")
print("========================================================")


📡 요청 시도: 2015년 01월
✅ **성공.** HTTP 200, JSON 파싱 완료. 수집 데이터 100건 확인.

📡 요청 시도: 2015년 02월
✅ **성공.** HTTP 200, JSON 파싱 완료. 수집 데이터 100건 확인.

📡 요청 시도: 2015년 03월
✅ **성공.** HTTP 200, JSON 파싱 완료. 수집 데이터 100건 확인.

📡 요청 시도: 2015년 04월
✅ **성공.** HTTP 200, JSON 파싱 완료. 수집 데이터 100건 확인.

📡 요청 시도: 2015년 05월
✅ **성공.** HTTP 200, JSON 파싱 완료. 수집 데이터 100건 확인.

📡 요청 시도: 2015년 06월
✅ **성공.** HTTP 200, JSON 파싱 완료. 수집 데이터 100건 확인.

📡 요청 시도: 2015년 07월
✅ **성공.** HTTP 200, JSON 파싱 완료. 수집 데이터 100건 확인.

📡 요청 시도: 2015년 08월
✅ **성공.** HTTP 200, JSON 파싱 완료. 수집 데이터 100건 확인.

📡 요청 시도: 2015년 09월
✅ **성공.** HTTP 200, JSON 파싱 완료. 수집 데이터 100건 확인.

📡 요청 시도: 2015년 10월
✅ **성공.** HTTP 200, JSON 파싱 완료. 수집 데이터 100건 확인.

📡 요청 시도: 2015년 11월
✅ **성공.** HTTP 200, JSON 파싱 완료. 수집 데이터 100건 확인.

📡 요청 시도: 2015년 12월
✅ **성공.** HTTP 200, JSON 파싱 완료. 수집 데이터 100건 확인.

📡 요청 시도: 2016년 01월
✅ **성공.** HTTP 200, JSON 파싱 완료. 수집 데이터 100건 확인.

📡 요청 시도: 2016년 02월
✅ **성공.** HTTP 200, JSON 파싱 완료. 수집 데이터 100건 확인.

📡 요청 시도: 2016년 03월
✅ **성공.** HTTP 200, JSON 파싱 